In [ ]:
!python3 -m pip install "fastapi[all]"

In [20]:
%%writefile main.py

import uvicorn
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from http import HTTPStatus
from typing import Dict, List, Union, Optional
from enum import Enum

from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from statsmodels.tsa.arima.model import ARIMA
from fastapi.encoders import jsonable_encoder


app = FastAPI(
    docs_url="/api/openapi",
    openapi_url="/api/openapi.json",
)


class Hyperparameters(BaseModel):
    optimized: Optional[bool]

class ModelTypeEnum(str, Enum):
  ses = 'ses'
  arima = 'arima'

class ModelConfig(BaseModel):
  hyperparameters: Hyperparameters
  id: str
  ml_model_type: ModelTypeEnum

class ForecastRequest(BaseModel):
  X: List[List[float]]
  config: ModelConfig
  steps: int

class ForecastResponse(BaseModel):
  id: str
  predictions: List[float]

class LoadRequest(BaseModel):
  id: str

class LoadResponse(BaseModel):
  message: str

class ModelListResponse(BaseModel):
  id: str

class RemoveResponse(BaseModel):
  message: str


models = {}
inf_models = {}

def ses_forecast(train, steps):
  preds = []
  cur = train['Price'].copy()
  idx = cur.index[-1]

  for _ in range(steps):
      model = SimpleExpSmoothing(cur).fit(optimized=True)
      alpha = model.params['smoothing_level']  # Получаем значение alpha
      print(f"Параметр SES: alpha={alpha:.4f}")

      f = model.forecast(1).iloc[-1]

      lo = f * 0.95
      hi = f * 1.05
      preds.append((f, lo, hi))

      idx += pd.Timedelta(days=1)
      cur.loc[idx] = f

  return preds

def arima_forecast(a,b):
  pass


@app.post("/forecast", response_model=list[ForecastResponse], status_code=HTTPStatus.CREATED)
async def fit(request:list[ForecastRequest]):
  list_of_inputs = jsonable_encoder(request)
  for input in list_of_inputs:
    if input['config']['ml_model_type'] == 'ses':
      preds = ses_forecast(input['X'], input['steps'])
    else:
      preds = arima_forecast(input['X'], input['steps'])
  return [{'id': input['id'], 'predictions': preds}]


@app.post("/load", response_model=list[LoadResponse])
async def load(request: LoadRequest):
  input = jsonable_encoder(request)
  global inf_models
  inf_models[input['id']] = models[input['id']]
  return [{'message': f"Model '{input['id']}' loaded"}]


@app.get("/list_models", response_model=list[ModelListResponse])
async def list_models():
  res = []
  for k in models.keys():
    res.append({'id': k})
  return res


@app.delete("/remove_all", response_model=list[RemoveResponse])
async def remove_all():
  global models
  res = []
  for k in models.keys():
    res.append({'message': f"Model '{k}' removed"})
  models = []
  return res


if __name__ == "__main__":
    uvicorn.run("main:app", host="0.0.0.0", port=8000, reload=True)

Overwriting main.py


In [23]:
!curl https://loca.lt/mytunnelpassword

35.204.224.125

In [7]:
!npm install -g localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧
added 22 packages in 3s
⠧
⠧3 packages are looking for funding
⠧  run `npm fund` for details
⠧

In [24]:
!uvicorn main:app & npx localtunnel --port 8000 --subdomain fastapi & wget -q -O - https://loca.lt/mytunnelpassword

⠙⠹⠸⠼⠴⠦⠧⠇⠏your url is: https://fastapi.loca.lt
INFO:     Started server process [36844]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     85.140.5.120:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     85.140.5.120:0 - "GET /api/openapi HTTP/1.1" 200 OK
INFO:     85.140.5.120:0 - "GET /api/openapi.json HTTP/1.1" 200 OK
INFO:     85.140.5.120:0 - "POST /forecast HTTP/1.1" 500 Internal Server Error
ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/protocols/http/httptools_impl.py", line 409, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/fastapi/applications.py", line 1054, in